# The GRAD-GPAD framework 🗿 
    
    ➡️ Foundations: The Python Library
---

The `gradgpad` framework povide python modules to ease face-PAD research. This tutorial is designed to help you become familiar with the package, undertanding the **Foundations** of the package.


![gradgpad_detailed_architecture](images/gradgpad_detailed_architecture.jpeg "GRAD-GPAD Architecture")



## Table of Contents 👩‍💻
- Installation 💻
- Environment
- Annotations
- Scores
- Results
- Metrics 

### Installation 💻

In [ ]:
pip install -U gradgpad

---
### Environment

We start importing required objects:

In [2]:
import json
from gradgpad import (
    annotations,
    calculate_pai_stats,
    ScoresProvider,
    Approach,
    Protocol,
    Subset,
    Dataset,
    Device,
    CoarseGrainPai,
    Filter,
    Sex,
    Age,
    SkinTone,
    Scenario,
    Metrics,
    MetricsDemographics,
    ResultsProvider
)

---
### Annotations

There are nearly 30000 annotations for the 13 datasets presented in the GRAD-GPAD v2.

In [11]:
assert annotations.num_annotations == 29567

You can print each of the annotations using sorted index. Note if not select an annotation index it will print the `29567` annotations, but this is not recommended to use on a Notebook.

In [12]:
print(annotations.annotations[0])
annotations.print_semantic(annotation_index=0)

Annotation: {
    "attributes": {
        "conditions": {
            "capture_device": 0,
            "lighting": 0
        },
        "person": {
            "age": 1,
            "sex": 0,
            "skin_tone": 4
        }
    },
    "dataset": "casia-fasd",
    "id": "casia-fasd_test_release/1/6.avi",
    "media": "test_release/1/6.avi",
    "spai": {
        "classical": 3,
        "specific": 7,
        "type": 1
    }
}
{
    "annotations": [
        {
            "attributes": {
                "conditions": {
                    "capture_device": "webcam low quality",
                    "lighting": "controlled"
                },
                "person": {
                    "age": "adult",
                    "sex": "male",
                    "skin_tone": "medium yellow brown"
                }
            },
            "dataset": "casia-fasd",
            "id": "casia-fasd_test_release/1/6.avi",
            "media": "test_release/1/6.avi",
            "spai": {
     

You can also, filter by dataset ids as the example below:

In [13]:
selected_ids = [
    "replay-mobile_devel/real/client023_session02_authenticate_tablet_controlled.mov", 
    "replay-mobile_devel/real/client025_session02_authenticate_tablet_controlled.mov",
    "replay-mobile_devel/real/client029_session02_authenticate_mobile_direct.mov",
    "replay-mobile_devel/real/client005_session02_authenticate_mobile_controlled.mov"]
selected_annotations = annotations.get_annotations_from_ids(selected_ids)
assert len(selected_annotations) == len(selected_ids)

Calculate Presentation Attack Intrument Statics from `annotations` with `calculate_pai_stats` function.

In [14]:
gradgpad_pai_stats = calculate_pai_stats(annotations)
print(json.dumps(gradgpad_pai_stats, indent=4, sort_keys=True))

{
    "num_attacks": 23663,
    "num_coarse_grained_pai": {
        "makeup": 135,
        "mask": 1781,
        "partial": 773,
        "print": 4290,
        "replay": 16684
    },
    "num_fine_grained_pai": {
        "makeup cosmetic": 51,
        "makeup impersonation": 61,
        "makeup obfuscation": 23,
        "mask paper": 766,
        "mask rigid": 829,
        "mask silicone": 186,
        "partial funny eyes": 160,
        "partial lower half": 29,
        "partial paper glasses": 127,
        "partial periocular": 57,
        "partial upper half": 400,
        "print high quality": 2484,
        "print low quality": 1390,
        "print medium quality": 416,
        "replay high quality": 8491,
        "replay low quality": 400,
        "replay medium quality": 7793
    },
    "num_genuines": 5904,
    "num_type_pai": {
        "type_1": 22816,
        "type_2": 486,
        "type_3": 361
    },
    "percentage_num_coarse_grained_pai": {
        "makeup": 0.5705109242276

This stadistic are reflected on work [CITATION] Figure XXXX

<img src="images/grad-gpad-pais.png" width="300">

---
### Scores

The `gradgpad` library has available a class to provide scores (`ResultProvider`).
To retrieve all scores for a specific approach, use `ScoresProvider.all(approach)`

In [15]:
scores_quality = ScoresProvider.all(Approach.QUALITY_RBF)
scores_auxiliary = ScoresProvider.all(Approach.AUXILIARY)

To retrieve all scores by subsets, use `ScoresProvider.get_subsets(approach, protocol)`:

In [16]:
scores_auxiliary_subsets = ScoresProvider.get_subsets(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST
)
scores_auxiliary_devel = scores_auxiliary_subsets.get("devel")
scores_auxiliary_test = scores_auxiliary_subsets.get("test")

Besides, you can obtain scores with `ScoresProvider.get(approach, protocol, subset, dataset, device, pai)`

In [17]:
scores_auxiliary_test = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST)

Or more specific scores

In [18]:
scores_specific = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST, 
    Dataset.REPLAY_MOBILE, 
    Device.MOBILE_TABLET, 
    CoarseGrainPai.REPLAY
)

You can check every option with static method `options`

In [19]:
print(f"- Approaches:\n  > {Approach.options()}\n")
print(f"- Protocol:\n  > {Protocol.options()}\n")
print(f"- Subset:\n  > {Subset.options()}\n")
print(f"- Dataset:\n  > {Dataset.options()}\n")
print(f"- CoarseGrainPai:\n  > {CoarseGrainPai.options()}\n")

- Approaches:
  > [<Approach.QUALITY_RBF: 'quality_rbf'>, <Approach.QUALITY_RBF_BALANCED: 'quality_rbf_balanced'>, <Approach.QUALITY_LINEAR: 'quality_linear'>, <Approach.AUXILIARY: 'auxiliary'>, <Approach.CONTINUAL_LEARNING_AUXILIARY: 'continual_learning_auxiliary'>]

- Protocol:
  > [<Protocol.GRANDTEST: 'grandtest'>, <Protocol.GRANDTEST_SEX_50_50: 'grandtest_sex_50_50'>, <Protocol.GRANDTEST_SEX_80_20: 'grandtest_sex_80_20'>, <Protocol.GRANDTEST_SEX_90_10: 'grandtest_sex_90_10'>, <Protocol.CROSS_DATASET: 'cross_dataset'>, <Protocol.CROSS_DEVICE: 'cross_device'>, <Protocol.LODO: 'lodo'>, <Protocol.UNSEEN_ATTACK: 'unseen_attack'>, <Protocol.INTRADATASET: 'intradataset'>]

- Subset:
  > [<Subset.DEVEL: 'devel'>, <Subset.TEST: 'test'>]

- Dataset:
  > [<Dataset.CASIA_FASD: 'casia-fasd'>, <Dataset.THREEDMAD: 'threedmad'>, <Dataset.UVAD: 'uvad'>, <Dataset.SIW_M: 'siw-m'>, <Dataset.SIW: 'siw'>, <Dataset.ROSE_YOUTU: 'rose-youtu'>, <Dataset.REPLAY_MOBILE: 'replay-mobile'>, <Dataset.REPLAY_ATTA

Additionaly, provided `Scores` class implements useful tools to manage obtained scores from an experiment: 

In [21]:
scores = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST
)

genuine_scores = scores.get_genuine()
genuine_attacks = scores.get_attacks()

Using `numpy` for data management in your experiments, just use `get_numpy_scores` and `get_numpy_labels`

In [22]:
numpy_scores_genuine = scores.get_numpy_scores()
numpy_scores_labels = scores.get_numpy_labels()

Another interesting tool that implements `Scores` object is the filtering method `filtered_by` wich uses `Filter` object.

Imagine we want filter by `Sex`, `Age` or `SkinTone`:

In [23]:
# Sex
male_scores = scores.filtered_by(Filter(sex=Sex.MALE))
female_scores = scores.filtered_by(Filter(sex=Sex.FEMALE))

# Age
young_scores = scores.filtered_by(Filter(age=Age.YOUNG))
adult_scores = scores.filtered_by(Filter(age=Age.ADULT))
senior_scores = scores.filtered_by(Filter(age=Age.SENIOR))

# SkinTone
light_pink_scores = scores.filtered_by(Filter(skin_tone=SkinTone.LIGHT_PINK))
light_yellow_scores = scores.filtered_by(Filter(skin_tone=SkinTone.LIGHT_YELLOW))
medium_pink_brown_scores = scores.filtered_by(Filter(skin_tone=SkinTone.MEDIUM_PINK_BROWN))
medium_dark_brown_scores = scores.filtered_by(Filter(skin_tone=SkinTone.MEDIUM_DARK_BROWN))
dark_brown_scores = scores.filtered_by(Filter(skin_tone=SkinTone.DARK_BROWN))

Or just filter by dataset

In [24]:
casia_fasd_scores = scores.filtered_by(Filter(dataset=Dataset.CASIA_FASD))
threedmad_scores = scores.filtered_by(Filter(dataset=Dataset.THREEDMAD))
uvad_scores = scores.filtered_by(Filter(dataset=Dataset.UVAD))
siw_m_scores = scores.filtered_by(Filter(dataset=Dataset.SIW_M))
siw_scores = scores.filtered_by(Filter(dataset=Dataset.SIW))
rose_youtu_scores = scores.filtered_by(Filter(dataset=Dataset.ROSE_YOUTU))
replay_mobile_scores = scores.filtered_by(Filter(dataset=Dataset.REPLAY_MOBILE))
replay_attack_scores = scores.filtered_by(Filter(dataset=Dataset.REPLAY_ATTACK))
oulu_npu_scores = scores.filtered_by(Filter(dataset=Dataset.OULU_NPU))
msu_mfsd_scores = scores.filtered_by(Filter(dataset=Dataset.MSU_MFSD))
hkbu_v2_scores = scores.filtered_by(Filter(dataset=Dataset.HKBU_V2))
hkbu_scores = scores.filtered_by(Filter(dataset=Dataset.HKBU))
csmad_scores = scores.filtered_by(Filter(dataset=Dataset.CSMAD))

Or by scenario:
* **Genuine** (`Scenario.GENUINE`): Genuine attemps;
* **PAS I** (`Scenario.PAS_TYPE_I`): represents a scenario where spoofers have freedom to try to impersonate an identity completely (as with a stolen cell phone or in an isolated access environment);
* **PAS II** (`Scenario.PAS_TYPE_II`): represents partial identity impersonations, where attackers only use a part of someone else’s identity;
* **PAS III** (`Scenario.PAS_TYPE_III`): where users try to hide their identity without impersonating anyone in particular

In [25]:
genuine_scores = scores.filtered_by(Filter(scenario=Scenario.GENUINE))
pas_I_scores = scores.filtered_by(Filter(scenario=Scenario.PAS_TYPE_I))
pas_II_scores = scores.filtered_by(Filter(scenario=Scenario.PAS_TYPE_II))
pas_III_scores = scores.filtered_by(Filter(scenario=Scenario.PAS_TYPE_III))

Finally, `Scores` class provides you fair subsets (balanced) for demographic labels.

In [26]:
sex_fair_subset = scores.get_fair_sex_subset()
age_fair_subset = scores.get_fair_age_subset()
skin_tone_fair_subset = scores.get_fair_skin_tone_subset()

---
### Metrics

In [17]:
scores_auxiliary_devel = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.DEVEL
)

scores_auxiliary_test = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST
)

metrics = Metrics(
    devel_scores=scores_auxiliary_devel,
    test_scores=scores_auxiliary_test
)

print(f"AUXILIARY EER [Devel: {metrics.get_eer(Subset.DEVEL)} | Test: {metrics.get_eer(Subset.TEST)}]")
print(f"AUXILIARY EER Th [Devel: {metrics.get_eer_th(Subset.DEVEL)} | Test: {metrics.get_eer_th(Subset.TEST)}]")

AUXILIARY EER [Devel: 0.09306409130803167 | Test: 0.0826664064032106]
AUXILIARY EER Th [Devel: 0.5883721313106811 | Test: 0.6490925550464677]


---
### Results

`ResultsProvider` is implemented on top on scores and is a new level of abstraction to obtain results from Scores and Metric calculators.

In [27]:
results_auxiliary_grandtest = ResultsProvider.grandtest(Approach.AUXILIARY)
results_auxiliary_cross_dataset = ResultsProvider.cross_dataset(Approach.AUXILIARY)
results_auxiliary_lodo = ResultsProvider.lodo(Approach.AUXILIARY)
results_auxiliary_cross_device = ResultsProvider.cross_device(Approach.AUXILIARY)
results_auxiliary_unseen_attack = ResultsProvider.unseen_attack(Approach.AUXILIARY)

If you print the result dictionary, you can check detailed result report. This function loads the scores and calculate several working points for default face-PAD metrics.

In [28]:
print(json.dumps(results_auxiliary_grandtest, indent=4, sort_keys=True))

{
    "grandtest": {
        "aggregate": {
            "acer": 28.433285123272007,
            "acer_per_pai": {
                "print": 8.01738412073659,
                "replay": 28.433285123272007
            },
            "apcer_per_pai": {
                "print": 5.3220441487753245,
                "replay": 46.15384615384615
            },
            "apcer_per_pai_fixing_bpcer": {
                "print": {
                    "apcer_fixing_bpcer1": 25.370426368309644,
                    "apcer_fixing_bpcer10": 5.53371635923798,
                    "apcer_fixing_bpcer15": 3.41699425461143,
                    "apcer_fixing_bpcer20": 2.237677653462353,
                    "apcer_fixing_bpcer25": 1.6732184255619393,
                    "apcer_fixing_bpcer30": 1.1591573430097772,
                    "apcer_fixing_bpcer35": 0.806370325572019,
                    "apcer_fixing_bpcer40": 0.554379598830763,
                    "apcer_fixing_bpcer45": 0.3729462755770588,
         

You also can use `ResultsProvider.all` to calculate each available protocol.

In [29]:
results_auxiliary = ResultsProvider.all(Approach.AUXILIARY)

Additionally, `ResultsProvider` can calculate results for evenly demographic distributed subsets for the Grandtest Protocol:

TODO: Review!!!!!

In [8]:
results_auxiliary_grandtest_sex = ResultsProvider.get_grandtest_sex(Approach.AUXILIARY)
results_auxiliary_grandtest_age = ResultsProvider.get_grandtest_age(Approach.AUXILIARY)
results_auxiliary_grandtest_skin_tone = ResultsProvider.get_grandtest_skin_tone(Approach.AUXILIARY)

TypeError: float() argument must be a string or a number, not 'dict'